In [ ]:
!git clone https://github.com/UH-Insure/Evaluation.git
%cd Evaluation
!ls -la

# Single-Model Evaluation (Cryptol/SAW)

Choose Model A for Josh's model, Model B for Talha's model

Pipeline: install SAW/Cryptol → load tasks → generate k candidates → compile with `cryptol` → optional verify with `saw` → metrics/pass@k.


In [ ]:
!git clone https://github.com/UH-Insure/crypto-c.git
%cd crypto-c
!chmod +x scripts/colab_setup.sh
!bash scripts/colab_setup.sh
!bash scripts/run-all-saw.sh
%cd ..

In [ ]:
MODEL_A = "j05hr3d/peft-FT-3-Coder-30b-v3"  
MODEL_B = "tam2003/peft-FT-3-Coder-30b-v3"  
CHOOSE = "A"  
CUSTOM  = "" 
TASKS_PATH = "eval/prompts/tasks.jsonl" 
K = 5 
MAX_NEW_TOKENS_DEFAULT = 512 
SEED = 42  
USE_DRIVE = False  

def pick(ch, a, b, c):
  if ch == "A": return a
  if ch == "B": return b
  return c.strip() or a

MODEL_ID = pick(CHOOSE, MODEL_A, MODEL_B, CUSTOM)
print("[+] Using model:", MODEL_ID)


In [ ]:

import os
if USE_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')
  BASE_OUT = '/content/drive/MyDrive/uh-crypto-llm-eval'
else:
  BASE_OUT = '../outputs'
os.makedirs(BASE_OUT, exist_ok=True)
print("[+] Outputs ->", BASE_OUT)


In [ ]:
!pip -q install --upgrade transformers accelerate datasets pandas tiktoken orjson
!bash ../scripts/install_saw_cryptol.sh
!saw --version || true
!cryptol --version || true


In [ ]:
import json, pathlib, subprocess, uuid, random
from datetime import datetime

random.seed(42)

def load_tasks(path:str):
  tasks=[]
  with open(path,'r') as f:
    for line in f:
      if line.strip(): tasks.append(json.loads(line))
  return tasks

def now_tag(): return datetime.utcnow().strftime('%Y%m%d_%H%M%S')

def try_compile_with_cryptol(code:str)->bool:
  tmp = pathlib.Path('tmp_cryptol'); tmp.mkdir(exist_ok=True)
  p = tmp / f"gen_{uuid.uuid4().hex}.cry"
  p.write_text(code, encoding='utf-8')
  cmd = ["bash","-lc", f"echo ':l {p}' | cryptol"]
  try:
    out = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=60)
    return out.returncode == 0 and b"Type checked." in out.stdout
  except Exception:
    return False

def run_harness_if_any(h)->bool:
  if not h: return False
  if not pathlib.Path(h).exists(): return False
  try:
    out = subprocess.run(["bash","-lc", f"saw {h}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=300)
    return out.returncode == 0
  except Exception:
    return False


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16, device_map="auto")
print("[+] Model loaded.")


In [ ]:
def generate_code(prompt:str, max_new_tokens:int=512, temperature:float=0.7, top_p:float=0.95, do_sample:bool=True):
  input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to(model.device)
  with torch.no_grad():
    out = model.generate(
      input_ids,
      max_new_tokens=max_new_tokens,
      temperature=temperature,
      top_p=top_p,
      do_sample=do_sample,
      pad_token_id=tokenizer.eos_token_id,
    )
  text = tokenizer.decode(out[0], skip_special_tokens=True)
  return text[len(prompt):].strip()


In [ ]:
import os, json
tasks = load_tasks(TASKS_PATH)
tag = now_tag()
outdir = os.path.join(BASE_OUT, tag)
os.makedirs(outdir, exist_ok=True)

raw_path = os.path.join(outdir, 'raw_generations.jsonl')
metrics_path = os.path.join(outdir, 'metrics.json')

total = compiles = verified = passk_compile = passk_verify = 0

with open(raw_path, 'w') as gf:
  for t in tasks:
    k = int(t.get('k', K))
    compiled_any = False
    verified_any = False
    for i in range(k):
      total += 1
      code = generate_code(t['prompt'], max_new_tokens=int(t.get('max_new_tokens', MAX_NEW_TOKENS_DEFAULT)))
      ok_compile = try_compile_with_cryptol(code)
      if ok_compile:
        compiles += 1; compiled_any = True
      ok_verify = False
      if t.get('harness'):
        ok_verify = run_harness_if_any(t['harness'])
        if ok_verify:
          verified += 1; verified_any = True
      row = {"task_id": t["task_id"], "attempt": i, "model": MODEL_ID, "prompt": t["prompt"], "code": code, "compiled": ok_compile, "verified": ok_verify}
      gf.write(json.dumps(row) + "\n")
    if compiled_any: passk_compile += 1
    if verified_any: passk_verify += 1

metrics = {
  "total_attempts": total,
  "compile_successes": compiles,
  "verify_successes": verified,
  "compile_rate": compiles/total if total else 0.0,
  "verify_rate": verified/total if total else 0.0,
  "pass@k_compile": passk_compile/len(tasks) if tasks else 0.0,
  "pass@k_verify": passk_verify/len(tasks) if tasks else 0.0,
  "model": MODEL_ID,
  "timestamp": tag,
}
with open(metrics_path, 'w') as mf: mf.write(json.dumps(metrics))
metrics


### Notes
- Point `harness` in `tasks.jsonl` to crypto-c SAW scripts.
- Keep SAW/Cryptol versions aligned with Docker for reproducibility.